In [1]:
!pip install regex huggingface_hub

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `BLOGAI` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `BLOGAI`


In [3]:
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
def segment_script(script_text: str):
    pattern = r"(?i)(?=\b(INT\.|EXT\.)\b)"
    scenes = re.split(pattern, script_text)

    scene_list = []
    current_scene = ""
    for token in scenes:
        if re.match(r"(?i)^(INT\.|EXT\.)", token.strip()):
            if current_scene:
                scene_list.append(current_scene.strip())
            current_scene = token.strip()
        else:
            current_scene += " " + token.strip()
    if current_scene:
        scene_list.append(current_scene.strip())
    return scene_list

In [6]:
def generate_scene_story(scene: str, style: str = "default") -> str:
    prompt = (
        f"Transform the following screenplay scene into a detailed, immersive narrative with a {style} tone. "
        "Focus on character emotions, setting details, and internal monologues to bring the scene to life:\n\n"
        f"{scene}\n\n"
        "Narrative:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=1024, do_sample=True, temperature=0.8)
    story_segment = tokenizer.decode(output[0], skip_special_tokens=True)
    return story_segment.strip()

In [7]:
def generate_full_story(script: str, style: str = "default") -> str:
    scenes = segment_script(script)
    if not scenes:
        return "No scenes detected. Please check the script format."

    narrative_segments = []
    for idx, scene in enumerate(scenes):
        print(f"Processing Scene {idx+1}/{len(scenes)}...")
        segment_story = generate_scene_story(scene, style=style)
        narrative_segments.append(f"Scene {idx+1}:\n{segment_story}\n")
    full_story = "\n".join(narrative_segments)
    return full_story

In [8]:
def split_after_narrative(text: str) -> str:
    split_token = "Narrative:"
    idx = text.find(split_token)
    if idx != -1:
        return text[idx + len(split_token):].strip()
    else:
        return text


In [22]:
def unload_model():
    global model, tokenizer
    del model
    del tokenizer
    torch.cuda.empty_cache()
    print("Model and tokenizer unloaded, GPU cache cleared.")

In [17]:
if __name__ == "__main__":
    example_script = """
    INT. COFFEE SHOP – DAY
    Alice sits by the window, staring at her phone. Bob enters hesitantly and waves.
    BOB
    (nervous)
    Hi, Alice.
    ALICE
    (distant)
    Hi, Bob.

    EXT. CITY STREET – EVENING
    The city buzzes with life as cars and pedestrians move along the busy avenue.
    A street musician plays a soulful tune on his guitar, capturing the attention of passersby.
    """

    print("📜 Input Script:\n", example_script)
    print("\nGenerating full narrative story. Please wait...\n")
    final_story = generate_full_story(example_script, style="dramatic")
    final_story = split_after_narrative(final_story)
    print("\n📖 Generated Story:\n", final_story)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📜 Input Script:
 
    INT. COFFEE SHOP – DAY
    Alice sits by the window, staring at her phone. Bob enters hesitantly and waves.
    BOB
    (nervous)
    Hi, Alice.
    ALICE
    (distant)
    Hi, Bob.

    EXT. CITY STREET – EVENING
    The city buzzes with life as cars and pedestrians move along the busy avenue.
    A street musician plays a soulful tune on his guitar, capturing the attention of passersby.
    

Generating full narrative story. Please wait...

Processing Scene 1/1...

📖 Generated Story:
 Alice's heart pounded in her chest as she sat hunched over her phone at the coffee shop window. The sunlight streaming through the panes of glass illuminated her anxious features, casting deep shadows across her face. The cacophony of clinking dishes and murmured conversations from the coffee shop seemed to fade into oblivion, leaving her entirely alone with her thoughts.

Bob shuffled his feet on the worn wooden floorboards, hesitant to disturb her solitude. With each deliberate s

In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00


In [11]:
import gradio as gr

def create_interface():
    def process_script(script, style, progress=gr.Progress()):
        progress(0, desc="Initializing...")

        progress(0.2, desc="Segmenting scenes...")
        scenes = segment_script(script)

        full_story = []
        for idx, scene in enumerate(scenes):
            progress((idx+1)/len(scenes), desc=f"Processing Scene {idx+1}/{len(scenes)}")
            segment = generate_scene_story(scene, style)
            clean_segment = split_after_narrative(segment)
            full_story.append(f"SCENE {idx+1}:\n{clean_segment}")

        progress(1.0, desc="Finalizing...")
        return "\n\n".join(full_story)

    with gr.Blocks(theme=gr.themes.Soft(), title="Script-Symphony") as interface:
        gr.Markdown("# 🎬 Script-Symphony")
        gr.Markdown("From Screen to Sound—Unfold Your Story into an Epic Narrative!")

        with gr.Row():
            with gr.Column(scale=2):
                input_script = gr.Textbox(
                    label="Screenplay Input",
                    placeholder="Paste your screenplay here...",
                    lines=15,
                    elem_id="input-box"
                )
                style_select = gr.Dropdown(
                    choices=["Dramatic", "Suspenseful", "Romantic", "Neutral"],
                    value="Dramatic",
                    label="Narrative Style"
                )
                submit_btn = gr.Button("Convert to Novel", variant="primary")

            with gr.Column(scale=3):
                output_story = gr.Textbox(
                    label="Generated Narrative",
                    lines=20,
                    interactive=False,
                    elem_id="output-box"
                )

        examples = gr.Examples(
            examples=[[
                """INT. COFFEE SHOP - DAY
Alice fidgets with her cup. Bob enters, looking anxious.""",
                "Dramatic"
            ]],
            inputs=[input_script, style_select]
        )

        submit_btn.click(
            fn=process_script,
            inputs=[input_script, style_select],
            outputs=output_story
        )

        clear_btn = gr.ClearButton([input_script, output_story])
        gr.Markdown("### Tips:\n- Use proper screenplay formatting with INT/EXT headings\n- Keep scenes under 500 words for best results\n- Adjust style for different emotional tones")

    return interface

if __name__ == "__main__":
    interface = create_interface()
    interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://505b3eaa7bbaa7654b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
